In [22]:
%cd ..

/home/haryoaw/documents/courses/nlp802/project/texteditalay


In [24]:
import pandas as pd
from neo_stif.components.train_data_preparation import prepare_data_tagging_and_pointer
import datasets
from torch.utils.data import DataLoader
from neo_stif.components.collator import FelixCollator
from neo_stif.components.utils import create_label_map
from transformers import BertForMaskedLM

In [4]:
df = pd.read_csv("../data/stif_indo/train_with_pointing.csv")

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")


/home/haryoaw/mambaforge/envs/sensei/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
idx = 433
tokenized_inp = tokenizer.tokenize(df.informal[idx], add_special_tokens=True)
print(len(tokenized_inp))
print(len(eval(df.label[idx])))

28
28


In [27]:
label_dict = create_label_map(30, True)
df_train = pd.read_csv("data/stif_indo/train_with_pointing.csv")
data_train = datasets.Dataset.from_pandas(df_train)
data_train, label_dict = prepare_data_tagging_and_pointer(data_train, tokenizer, label_dict)

Map: 100%|██████████| 1922/1922 [00:01<00:00, 1584.64 examples/s]


In [30]:
data_train

Dataset({
    features: ['informal', 'formal', 'point_indexes', 'label', 'len_label', 'informal_input_ids', 'informal_token_type_ids', 'informal_attention_mask', 'formal_input_ids', 'formal_token_type_ids', 'formal_attention_mask', 'tag_labels', 'point_labels'],
    num_rows: 1922
})

In [34]:
idx = 433
label_dict_reverse = {v: k for k, v in label_dict.items()}
tokenized_inp = tokenizer.tokenize(data_train['informal'][idx], add_special_tokens=True)
translated_label = [label_dict_reverse[label] for label in data_train['tag_labels'][idx]]
print(len(tokenized_inp))
print(len((data_train['tag_labels'][idx])))
print(list(zip(tokenized_inp, translated_label)))

28
28
[('[CLS]', 'KEEP'), ('betul', 'KEEP'), (',', 'KEEP'), ('terkait', 'KEEP'), ('cash', 'KEEP'), ('##back', 'KEEP'), ('.', 'KEEP'), ('terlihat', 'KEEP'), ('sepele', 'KEEP'), (',', 'KEEP'), ('namun', 'KEEP'), ('cara', 'KEEP'), ('xu', 'KEEP'), ('##ser', 'KEEP'), ('##x', 'KEEP|4'), ('mengh', 'DELETE'), ('##and', 'DELETE'), ('##len', 'DELETE'), ('##ya', 'DELETE'), ('yg', 'DELETE'), ('bikin', 'DELETE'), ('sakit', 'KEEP'), ('hati', 'KEEP|1'), ('&', 'DELETE'), ('sangat', 'KEEP'), ('mengecewakan', 'KEEP'), ('.', 'KEEP'), ('[SEP]', 'KEEP')]


In [35]:
loader = DataLoader(
    data_train,
    batch_size=2,
    shuffle=True,
    collate_fn=FelixCollator(tokenizer, pad_label_as_input=len(label_dict)),
)

In [36]:
after_load = next(iter(loader))

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [41]:
after_load['input_ids'][0])

'[CLS] ini sudah terkirim tp pesanan belum selesai d sistem karna saya menggunakan kurir berbeda [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'